In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('display.width', 500)
pd.set_option('display.notebook_repr_html', True)
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline
import math
from collections import defaultdict
from collections import Counter
from astropy.io import fits

import importlib
import ctypes

import ephem_forces

import importlib

Here we demonstrate the ephemeris-quality integrator by calling a python-wrapped C function that has been compiled into a library that is imported with the ephem_forces.py package.


The main function in ephem_forces is "integration_function".  It integrates massless test particles in the field of the Sun, planets, moon, and 16 massive asteroids.  It also includes the J2 and J4 gravitational harmonics of the Earth, the J2 gravitational harmonic of the Sun, and the solar GR terms (using the PPN formulation).  

The positions of the massive bodies come from two binary files, both from JPL.  The first is for the Sun, planets, and moon.  The other is for the asteroids.  It is important to know that the coordinate frame and units are not flexible.  The coordinate frame is the equatorial ICRF, which is the native coordinate system for the JPL binary files.  Note that this is equatorial rather than ecliptic.  In addition, the native coordinates are barycentric, rather than heliocentric.

For units we use solar masses, au, and days.  Furthermore, the independent time coordinate is TDB in Julian days.

integration_function is called with

tstart: the start time in JD (TDB)

tend: the end time in JD (TDB)

tstep: a suggested time step in days.  The integrator might alter this, depending upon the value of epsilon (see below).

geocentric: this is an integer (0 or 1).  0 is for barycentric and 1 is for geocentric.
n_particles: the integer number of input particles

instates: an array of 6-vectors, each of which is the position and velocity of a test particle at tstart.

invar_part: an array of integers that specify which real particle is the host for each input variational particle

invar: an array of 6-vectors, each of which represents a variational particle.

In [2]:
tstart, tstep, trange = 2458849.5, 20.0, 20
geocentric = 0
epsilon = 1e-8

In [3]:
tend = tstart + trange
tend = 2468851.011954478


In [4]:
# DE331
# (3666) Holman
# These are the equatorial, barycentric position and velocity (in AU and AU/day) at
# JD 2458849.5 TDB from the JPL Horizons website.
#row = [3.338875349745594E+00, -9.176518281675284E-01, -5.038590682977396E-01, 2.805663319000732E-03, 7.550408687780768E-03, 2.980028206579994E-03]

In [5]:
# DE441
# (3666) Holman
#2458849.500000000 = A.D. 2020-Jan-01 00:00:00.0000 TDB [del_T=     69.183900 s]
# X = 3.338875349745594E+00 Y =-9.176518281675284E-01 Z =-5.038590682977396E-01
# VX= 2.805663319000732E-03 VY= 7.550408687780768E-03 VZ= 2.980028206579994E-03
row = [3.338875349745594E+00, -9.176518281675284E-01, -5.038590682977396E-01, 2.805663319000732E-03, 7.550408687780768E-03, 2.980028206579994E-03]

Now we construct the input.  This will include a real particle with six associated variational particles, one for each phase space dimension.  It will include another six real particles, each offset by a small amount, given by 'scale', along each of those same dimension.  These will be used for testing purposes.

In [6]:
instates = np.array([row])
n_var = 6
n_particles = 7


invar_part = np.zeros(6, dtype=int)
invar = np.identity(6)

scale = 1e-8
instatesp = np.array([row]*6)+scale*invar
instates=np.vstack([instates, instatesp])


These are simpler initial conditions, without the additional real particles.

Now we integrate this one particle for 10,000 days.  The output is:

times: a numpy array of the times of output

states: a numpy array of 6-vectors, one for each real particle at each output time.

var: a numpy array of 6-vectors, one for each variational particle at each output time.

var_ng: a numpy array of vectors (length?) to represent the variations with respect to the non-gravitational parameters.

status: a flag indicating the outcome of the integration.


In [7]:
times, states, var, var_ng, status = ephem_forces.integration_function(tstart, tend, tstep, geocentric, 
                                                               n_particles, instates, n_var, invar_part, invar)#, epsilon=1e-8)


In [8]:
times[-1], states[-1][0]

(2468851.011954478,
 array([ 3.17645719e+00, -1.29320506e+00, -6.51569331e-01,  3.85018878e-03,
         7.18909588e-03,  2.79236764e-03]))

In [7]:
times[-1], states[-1][0]

(2468851.011954478,
 array([ 3.17644069e+00, -1.29323746e+00, -6.51581930e-01,  3.85028784e-03,
         7.18905495e-03,  2.79234744e-03]))

In [8]:
len(times), status

(3865, 0)

In [8]:
GMs = 132712440041.279419

In [9]:
au_km = 149597870.700
day_sec = 24*60*60.

In [10]:
GMsun = GMs*day_sec*day_sec/(au_km*au_km*au_km)
GMsun

0.00029591220828411956

In [11]:
GMm = 22031.868551
GMmerc = GMm*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMv = 324858.592000
GMven = GMv*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMe = 398600.435507
GMearth = GMe*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMmn = 4902.800118
GMmoon = GMmn*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMmr = 42828.375816
GMmars = GMmr*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMjup = 126712764.100000
GMjupiter = GMjup*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMsat = 37940584.841800
GMsaturn = GMsat*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMura = 5794556.400000
GMuranus = GMura*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMnep = 6836527.100580
GMneptune = GMnep*day_sec*day_sec/(au_km*au_km*au_km)

In [ ]:
GMpl = 975.500000
GMpluto = GMpl*day_sec*day_sec/(au_km*au_km*au_km)

In [13]:
print('%.16le' % (GMmerc))

4.9125001948001294e-11


### Other exploration

Each combination of initial conditional and associated parameters represent a trajectory over some time span, finite or infinite.  For the purposes of fitting an orbit to observations, it is necessary to determine the position, and possibly velocity, at the light-time corrected times of the observations.  Thus, it is often necessary to be able to determine the dynamical state at arbitrary times along the trajectory.  Rather than iteratively integrating to a set of times, it can be more efficient to integrate to a series of times and then interpolate the results for other times.

Note that for each overall step taken by the integrator output a number of substeps are included in the output.  I have explored a couple of options for this.  One is output at each of the Gauss-Radau integration substeps.  Another is output at the conventional Chebyshev nodes, as well as the end points.  Both sets of output make interpolation relatively easy, but the latter appears to have better performance.  The goal is to ensure that the error associated with the interpolation is smaller than the error of the integration itself, which is machine precision for IAS15.



In [ ]:
timesp, statesp, varp, varp_ng, statusp = ephem_forces.production_integration_function_wrapper(tstart, tend, epoch, n_particles, instates, epsilon=-1, tstep=40)


In [ ]:
import numpy.polynomial.chebyshev as ch

In [ ]:
pts = np.polynomial.chebyshev.chebpts1(7)
pts

In [ ]:
V = ch.chebvander(pts, 6 )

Vinv = np.linalg.inv(V)

In [ ]:
y = statesp[1:8,:,2]

In [ ]:
f0 = np.dot(Vinv, y)

In [ ]:
x = np.arange(-1, 1, 0.001)
x

In [ ]:
y0 = ch.chebval(x, f0)

In [ ]:
plt.plot(x, y0[0]-y[0])
#plt.ylim(-1e-15, 1e-15)

In [ ]:
y0, y

In [ ]:
timesp[0:9], timesp[4]

In [ ]:
timesp[8:17]

In [ ]:
(len(timesp)-1)/8

In [ ]:
i=1
np.reshape(statesp[i:i+9,:,0:3], (9, 3))

In [ ]:
%%time
fits = []
time_tags = []
for i in range(0, len(timesp)-1, 8):
    data = np.reshape(statesp[i:i+9,:,0:3], (9, 3))
    f = ch.chebfit(timesp[i:i+9]-timesp[i+4], data, 7)
    fits.append(f)
    time_tags.append(timesp[i])

In [ ]:
fits[0][:, 2]

In [ ]:
time_tags = np.array(time_tags)

In [ ]:
def find_segment(time_tags, t, tmax):
    if t < time_tags[0] or t > tmax:
        return -1
    else:
        idx = np.searchsorted(time_tags, t)
        return idx-1



In [ ]:
time_tags[-1]

In [ ]:
idx = find_segment(time_tags, 2458889.6, time_tags[-1]+40)
fits[idx]

In [ ]:
y = ch.chebval(x, f)

In [ ]:
x = 0.5*ch.chebpts1(7) + 0.5

In [ ]:
for v in x:
    print("%.16lf" %(v))

In [ ]:
model = [ch.chebval(t-timesp[4], f) for t in timesp[0:9]]


In [ ]:
plt.plot(timesp[0:9]-timesp[4], (statesp[0:9,:,2]-model))

In [ ]:
tend = tstart + trange
epoch = tstart + 0.5*trange

In [ ]:
times2, states2, var2, varp_ng2, statusp2 = ephem_forces.production_integration_function_wrapper(tstart, tend, epoch, n_particles, instates, epsilon=-1)


In [ ]:
len(times2), len(states2)

In [ ]:
times2, states2

In [ ]:
var_ng==None

### End Other Exploration

integration_function returns four items.   We  named them times, states, n_out,and n_particles.  

"times" is an array of the JD TDB times of each output.  

"states" is an array of each output state at each time.  It is organized first by time and then by state, with each state having six dimensions.  

"n_out" is the number of outputs.  This is somewhat redundant in python, because you can easily get the number of outputs from the length of "times" or the shape of "states".  

"n_particles" is the number of actual particles.

The underlying numerical integrator is IAS15 (Rein & Liu 2015), a 15th order predictor-corrector integrator with an adaptive step-size.  Each time step involves eight sub-steps.  We have modified the integrator to output the state at each of the sub-steps in order to support interpolation of the output.

Below is a plot of the overall step-size as a function of elapsed integration time.  (The sub-steps are smaller).  A rough periodicity on the ~2000 day asteroid orbital period is evident.

In [ ]:
t=(times-times[0])[::8]
dt=t[1:]-t[:-1]

plt.plot(t[:-1], dt)
plt.xlabel("time (days)")
plt.ylabel("step-size (days)")

In [ ]:
invar_part=np.array(6*[0]+6*[1])
invar = np.concatenate([np.identity(6), np.identity(6)])

And here is a histogram of the step-sizes.  Most sub-steps are 35-40 days.

In [ ]:
_=plt.hist(dt,bins=30)
plt.xlabel("step-size (days)")
plt.ylabel("N")

In [ ]:
dt.min()

Here are the xyz values as a function of time.  Keep in mind that the coordinate system is equatorial.

In [ ]:
for i in range(3):
    plt.plot(times-times[0], states[:,0,i])

In [ ]:
plt.plot(states[:,0,0], states[:,0,1], linewidth=0.2)
plt.axis('square')
plt.xlabel('x (AU)')
plt.ylabel('y (AU)')

And we can compare the output to what JPL Horizons gives.  We will first grab the time of the last output.

In [ ]:
times[-1]

In [ ]:
states[-1][0]

Next we grab the output from Horizons for the barycentric equatorial vectors.

In [9]:
# No non-gravs
# dt_min = 1e-2
# epsilon = 1e-8

# This is from JPL Horizons, when it used DE441 and sb431-n16
#2468851.011954478 = A.D. 2047-May-20 12:17:12.8669 TDB [del_T=     69.185185 s]
# X = 3.176457194815969E+00 Y =-1.293205057535584E+00 Z =-6.515693314430392E-01
# VX= 3.850188776785952E-03 VY= 7.189095881591583E-03 VZ= 2.792367641688407E-03

#DE431
holman = np.array([3.176457944594448E+00, -1.293204878669709E+00, -6.515691309014848E-01,
     3.850188579021247E-03, 7.189095526429492E-03, 2.792367993782722E-03])

#DE441
holman = np.array([3.176457194815969E+00, -1.293205057535584E+00, -6.515693314430392E-01, 3.850188776785952E-03, 7.189095881591583E-03, 2.792367641688407E-03])

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


The agreement after ~27 years is excellent, ~25 m or 1e-2 mas (assuming the object is 3 AU away).  This is probably due to the integrator or differences in the precision of the constants.


In [10]:
((states[-1][0]-holman)/3)*206265, (states[-1][0]-holman)*1.5e8

(array([-9.82105319e-06,  1.15228907e-05,  4.95372366e-06,  3.94689217e-08,
         8.67290368e-09,  1.50993994e-09]),
 array([-2.14261942e-02,  2.51390242e-02,  1.08073384e-02,  8.61077485e-05,
         1.89213228e-05,  3.29417483e-06]))

Let's look at the output states in more detail.

In [ ]:
times.shape, states.shape, n_particles

It looks like there are extra particles.  In fact, for each actual particle there are six "variational particles" or tangent vectors.  These are vectors with the same dimensionality as an actual particle state, but they are the result of integrating the linearized tangent equations, or variational equations.  There is one variational particle for each of the six dimensions, with the initial state being a unit vector.

One way to think of the variational equations is to consider two states that are initially close to each other.  As we integrate both the two states will begin to separate.  We can imagine a vector pointing from one of the particles to the other.  (This is a vector is all six dimensions, both positions and velocities.) 

Suppose that instead of integrating the two particles we could integrate one of the particles and the vector from that particle to the other.  In addition to the equations of motion for the actual particle, we would need the equations of motion for the state vector between the two.  

That is what the linearized variational equations are, the equations of motion for the separation between two particles.  As "linearized" suggests, these equations are good to first order in the separation.  Also, the variational equations are associated with a big 6x6 matrix, with the terms of the matrix depending only upon the state of the actual particle.  The terms in the matrix are independent of the state vector describing the separation of the particles.  

The variational equations are the result of multiplying this big matrix by the state vector of the current separation.  That means we can multiply the same big matrix by any number of state vectors.   The big matrix is sparse (most of the elements are zero). So, the multiplication is not too expensive.  


Let's demonstrate by integrating two actual particles and some tangent vectors.  We will offset the second particle by a small amount along the x-axis.

In [ ]:
states.shape, var.shape

The separation of the two actual particles and the tangent vector (appropriately scaled in length) coincide on the scale of the width of the lines.

In [ ]:
plt.plot(times-times[0], (states[:,2,:]-states[:,0,:])[:,0])

We can see the difference between the two approaches more clearly by subtracting one from the other.  They are, indeed, very close.  The difference is due to the nonlinear terms that are not included in the variational equations.

In [ ]:
plt.plot(times-times[0], (states[:,1,:]-states[:,0,:]-var[:,0,:]*scale)[:,0])
plt.plot(times-times[0], (states[:,2,:]-states[:,0,:]-var[:,1,:]*scale)[:,0])
plt.plot(times-times[0], (states[:,3,:]-states[:,0,:]-var[:,2,:]*scale)[:,0])
plt.xlabel("time (days)")
plt.ylabel("AU")

Now let's try some more challenging cases.  First Apophis, an NEO that makes repeated close approaches to Earth.  It will make an approach on 2021 Mar 06 1:06 UT.  This approach will not be particularly close.  However, the 2029 Apr 13 approach will be extremely close.

The initial conditions below are for 2020 Aug 01, before the first approach.  We will integrate through the first approach and to just before the closer second approach.

In [ ]:
importlib.reload(ephem_forces)

In [ ]:
row = [-5.145897476309183E-03, -7.554295792725090E-01, -2.803430954241811E-01, 1.994372392258838E-02, 2.695069501106252E-03, 1.506836811826654E-03]
    
tstart, tstep, trange = 2459062.5, 1.0, 3150
geocentric = 0
n_particles = 1
scale = 1e-8

del times
del states
instates = np.array(row)
times, states, n_out, n_particles = ephem_forces.integration_function(tstart, tstep, trange, geocentric, n_particles, instates)    

In [ ]:
times[-1], states[-1][0]


In [ ]:
apophis_before = np.array([
    -1.062610231926071E+00, -7.438995533507307E-03, -2.955702762445418E-02,
     1.995149470275391E-03, -1.426773516011195E-02, -5.250341507808811E-03])



# Including non-graves
# dt_min = 1e-3
# epsilon = 1e-8
apophis_before = np.array([
    -1.061749621090919E+00, -1.342851936299780E-02, -3.176051171408323E-02,
     2.104893542927364E-03, -1.426673477234808E-02, -5.247194855985309E-03])

# No non-gravs
apophis_before = np.array([
         -1.060823123742407E+00, -1.954485978603505E-02, -3.400944497155493E-02, 
          2.217117676737715E-03, -1.426507080914577E-02, -5.243738481667273E-03])


In [ ]:
states[-1][0], (states[-1][0]-apophis_before)*1.5e8

The largest discrepancy is ~119 km just before the 2029 close approach.

Now I will include the non-gravitational terms, using the A2 value provided by JPL Horizons (A1 and A3 = 0.0).

At the moment, the non-grav values are hard-coded.  The code needs to be recompiled and the library reloaded.

In [ ]:
importlib.reload(ephem_forces)

In [ ]:
row = [-5.145897476309183E-03, -7.554295792725090E-01, -2.803430954241811E-01, 1.994372392258838E-02, 2.695069501106252E-03, 1.506836811826654E-03]
    
tstart, tstep, trange = 2459062.5, 1.0, 3150
geocentric = 0
n_particles = 1
scale = 1e-8

del times
del states
instates = np.array(row)
times, states, n_out, n_particles = ephem_forces.integration_function(tstart, tstep, trange, geocentric, n_particles, instates)    

In [ ]:
times[-1]

In [ ]:
states[-1][0], (states[-1][0]-apophis_before)*1.5e8

In [ ]:
1e-3*24*3600

In [ ]:
row = [-5.145897476309183E-03, -7.554295792725090E-01, -2.803430954241811E-01, 1.994372392258838E-02, 2.695069501106252E-03, 1.506836811826654E-03]
    
tstart, tstep, trange = 2459062.5, 1.0, 3150
geocentric = 0
n_particles = 1
scale = 1e-8

del times
del states
instates = np.array(row)
times, states, n_out, n_particles = ephem_forces.integration_function(tstart, tstep, trange, geocentric, n_particles, instates)    

In [ ]:
row = [-5.145897476309183E-03, -7.554295792725090E-01, -2.803430954241811E-01, 1.994372392258838E-02, 2.695069501106252E-03, 1.506836811826654E-03]

rows = [[-5.145897476309183E-03, -7.554295792725090E-01, -2.803430954241811E-01, 1.994372392258838E-02, 2.695069501106252E-03, 1.506836811826654E-03],
       [-5.145898476309183E-03, -7.554295792725090E-01, -2.803430954241811E-01, 1.994372392258838E-02, 2.695069501106252E-03, 1.506836811826654E-03]]
   
   
tstart, tstep, trange = 2459062.5, 1.0, 3200
geocentric = 0
n_particles = 2
scale = 1e-8

instates = np.array(rows)
times, states, n_out, n_particles = ephem_forces.integration_function(tstart, tstep, trange, geocentric, n_particles, instates)    

In [ ]:
times[-1]

In [ ]:
(states[-1][0]-states[-1][1])*1.5e8, (states[0][0]-states[0][1])*1.5e8

In [ ]:
apophis_before = np.array([
    -1.062610231926071E+00, -7.438995533507307E-03, -2.955702762445418E-02,
     1.995149470275391E-03, -1.426773516011195E-02, -5.250341507808811E-03])


In [ ]:
tstart, tstep, trange = 2462213.958072528, 1.0, 2462263.697021989-2462216.753921871
geocentric = 0
n_particles = 1
scale = 1e-8

instates = apophis_before
times, states, n_out, n_particles = ephem_forces.integration_function(tstart, tstep, trange, geocentric, n_particles, instates)    

In [ ]:
times[-1]

In [ ]:
states[-1]

In [ ]:
apophis_after = np.array([
                 -6.283889415589419E-01, -6.496621274217913E-01, -2.652090553526072E-01,
                 1.554844044456806E-02, -1.021342630556588E-02, -3.747872168881012E-03])

apophis_after = np.array([
    -5.556667507595004E-01, -6.935286876694169E-01, -2.812079254765416E-01, 
    1.651530555748704E-02, -9.118721403508922E-03, -3.302633165414243E-03])

apophis_after = np.array([
    -6.398193089596965E-01, -6.420513159000456E-01, -2.624136502848963E-01,
    1.538287990494170E-02, -1.038167285986079E-02, -3.816576674477177E-03])


In [ ]:
states[-1][0], (states[-1][0]-apophis_after)*1.5e8

Now the largest discrepancy is ~14 km.  How carefully the very close approaches are handled matters, as expected.

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('display.width', 500)
pd.set_option('display.notebook_repr_html', True)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math
from collections import defaultdict
from collections import Counter
from astropy.io import fits

import importlib
import ctypes

import ephem_forces


Let's try geocentric integrations.

In [ ]:
row = [6.634500992578179E-02, 2.122458699845356E-03, -3.507415858744104E-04,
     -1.060257698567525E-03, 6.078332955992950E-04, 2.240712392485767E-04]

In [ ]:
tstart, tstep, trange = 2459062.5, -1.0, -400
geocentric = 1
n_particles = 1
scale = 1e-8

instates = np.array(row)
timesb, statesb, n_out, n_particles = ephem_forces.integration_function(tstart, tstep, trange, geocentric, n_particles, instates)  


In [ ]:
statesb

In [ ]:
plt.plot(statesb[:,0,0], statesb[:,0,1])

In [ ]:
del statesb

In [ ]:
tstart, tstep, trange = 2459062.5, 1.0, 400
geocentric = 1
n_particles = 1
scale = 1e-8
instates = np.array(row)

timesf, statesf, n_out, n_particles = ephem_forces.integration_function(tstart, tstep, trange, geocentric, n_particles, instates)

In [ ]:
statesf

In [ ]:
plt.plot(statesf[:,0,0], statesf[:,0,1])
plt.plot(statesb[:,0,0], statesb[:,0,1])

In [ ]:
tstart, tstep, trange = timesb[-1], 1.0, 400
instatesb = statesb[-1][0].copy()
timesf, statesf, n_out, n_particles = ephem_forces.integration_function(tstart, tstep, trange, geocentric, n_particles, instatesb)  

In [ ]:
plt.plot(statesb[:,0,0], statesb[:,0,1])

plt.plot(statesf[:,0,0], statesf[:,0,1])

In [ ]:
statesb

In [ ]:
len(states)

In [ ]:

2462263.697021989 = A.D. 2029-May-07 04:43:42.6999 TDB 
 X = 7.771821449378258E-02 Y = 1.444568330466675E-02 Z = 2.318152988506329E-02
 VX= 3.531021355764523E-03 VY= 8.992784904633009E-04 VZ= 1.064745760510192E-03



In [ ]:
apophis_geo_after=np.array([
    7.771821449378258E-02, 1.444568330466675E-02, 2.318152988506329E-02, 
    3.531021355764523E-03, 8.992784904633009E-04, 1.064745760510192E-03])


In [ ]:
(states[-1][0]-apophis_geo_after)*1.5e8

This is worse.  I wonder if this is due to the interpolated acceleration of the Earth.

In [ ]:
t=(times-times[0])[::8]
dt=t[1:]-t[:-1]

plt.plot(t[:-1], dt)
plt.xlabel("time (days)")
plt.ylabel("step-size (days)")

In [ ]:
np.min(dt)

In [ ]:
(30000/1.5e8)

In [ ]:
import demo_2particles

In [ ]:
demo_2particles.states[:,0,:]

In [ ]:
demo_2particles.times

In [ ]:
importlib.reload(demo_2particles)

In [ ]:
(1.9705228610653722e+00-1.970522858564949E+00)*1.5e8
(6.4568366227497698e-01-6.456836770230201E-01)*1.5e8

In [ ]:
demo_2particles.times.shape

In [ ]:
demo_2particles.states[:,0,:], demo_2particles.times

In [ ]:
dx= (demo.states[:,1,:]-demo.states[:,0,:])
tdx = demo.states[:,7,:]*1e-6

dy= (demo.states[:,2,:]-demo.states[:,0,:])
tdy = demo.states[:,8,:]*1e-6

dz= (demo.states[:,3,:]-demo.states[:,0,:])
tdz = demo.states[:,9,:]*1e-6



In [ ]:
tdx = demo.states[:,7,:]*1e-6
tdy = demo.states[:,8,:]*1e-6
tdz = demo.states[:,9,:]*1e-6



In [ ]:
tangents = demo.states[:,1:,:]

In [ ]:
tangents[1]

In [ ]:
#plt.plot(demo.times, (tdx - dx))
plt.plot(demo.times-demo.times[0], (tdx - dx))
plt.plot(demo.times-demo.times[0], (tdy - dy))
plt.plot(demo.times-demo.times[0], (tdz - dz))
#plt.xlim(0,200)
#plt.ylim(0, 10e-14)

In [ ]:
plt.plot(demo.times-demo.times[0], (demo.states[:,6,:]-demo.states[:,0,:])[:,0]-demo.states[:,12,:][:,0]*1e-6)
#plt.plot(demo.times-demo.times[0], demo.states[:,7,:][:,0]*1e-6)

In [ ]:
plt.plot(demo.states[:,0,:][:,0], demo.states[:,0,:][:,1])
plt.xlim(0, 4)
plt.ylim(-1, 3)

In [ ]:
del(demo.instates)
del(demo.times)
del(demo.states)

In [ ]:
import subprocess

In [ ]:
# Imports / set-up
import numpy as np ;
import os,sys ;
sys.path.append(os.environ['REBX_DIR']) ;
from examples.ephem_forces.ephem_forces import integration_function ;
tstart=2456117.641933589 ;
tstep=20 ;
trange=1000 ;
geocentric=False ;
n_particles=1 ;
reparsed_input=np.array([-2.0938349524664743,1.0009137200092553,0.41979849545335507,-0.004226738336365523, -0.009129140909705197, -0.0036271214539287102])


# Call that will randomly crash with malloc ...
integration_function(tstart, tstep, trange, geocentric,n_particles, reparsed_input)

In [ ]:
state = np.array((3.338876057509365E+00, -9.176517956664152E-01, -5.038590450387491E-01, 2.805663678557796E-03, 7.550408259144305E-03, 2.980028369986096E-03))
scales = np.logspace(-7, -3, 81)
results_dict = {}

for scale in scales:
    print('%12.3e' % scale)

    with open('test_ic', 'w') as f:
        f.write('tepoch 2458849.5\n')
        f.write('tstart 2458849.5\n')
        f.write('tstep 20.0\n')
        f.write('trange 500.\n')
        f.write('geocentric 0\n')

        f.write('state\n%23.16le %23.16le %23.16le\n%23.16le %23.16le %23.16le\n' % tuple(state))
        for v in state+np.identity(6)*scale:
            f.write('state\n%23.16le %23.16le %23.16le\n%23.16le %23.16le %23.16le\n' % tuple(v))

    output = subprocess.run(["./rebound", "test_ic", str(scale)], stdout=subprocess.PIPE)

    arr =np.loadtxt('out_states.txt')

    t = arr[:,0]

    px = arr[:,1:4]
    py = arr[:,7:10]
    pz = arr[:,13:16]
    
    results_dict[scale] = t, px, py, pz
    

In [ ]:
def plot_results(results):
    t, p0, p1, p2 = results
    d = np.linalg.norm(p0, axis=1)
    z = np.polyfit(t, d, deg=2)
    f = np.poly1d(z)
    plt.plot(t, d-f(t), label='x')
    #plt.plot(t, d, label='x')
    d = np.linalg.norm(p1, axis=1)
    z = np.polyfit(t, d, deg=2)
    f = np.poly1d(z)
    plt.plot(t, d-f(t), label='y')
    #plt.plot(t, d, label='y')
    d = np.linalg.norm(p2, axis=1)
    z = np.polyfit(t, d, deg=2)
    f = np.poly1d(z)
    plt.plot(t, d-f(t), label='z')
    #plt.plot(t, d, label='z')
    plt.legend()

In [ ]:
prec_x = np.array(sorted([(k, np.linalg.norm(p0[-1]), np.linalg.norm(p1[-1]), np.linalg.norm(p2[-1])) for k, (t, p0, p1, p2) in results_dict.items()]))



In [ ]:
plt.plot(prec_x[:,0], prec_x[:, 1])
plt.plot(prec_x[:,0], prec_x[:, 2])
plt.plot(prec_x[:,0], prec_x[:, 3])
plt.yscale('log')
plt.xscale ('log')

In [ ]:
plot_results(results_dict[1e-6])

In [ ]:
np.logspace(-8, -2, num=10)

In [ ]:
output.stdout

In [ ]:
np.linalg.solve(a, b)

In [ ]:
a = np.array([[1, -1, -1], [2, 3, 2], [4, 3, -2]])

In [ ]:
b = np.array([1, 8, -2])

In [ ]:
np.linalg.solve(a, b)

In [ ]:
a = np.array([[4, 6, -3], [3, 4, -6], [6, -3, 4]])

In [ ]:
b = np.array([24, 2, 46])

In [ ]:
np.linalg.solve(a, b)

In [ ]:
from ctypes import *

In [ ]:
libc = CDLL("libc.dylib")

In [ ]:
print(libc.time(None))  

In [ ]:
c_int()

In [ ]:
i = c_int(42)

In [ ]:
print(i)

In [ ]:
print(i.value)

In [ ]:
i.value=-99
print(i.value)

In [ ]:
s = "Hello, World"
c_s = c_wchar_p(s)
print(c_s)

In [ ]:
c_wchar_p(139966785747344)
print(c_s.value)

In [ ]:
print(c_s)

In [ ]:
c_s.value = "Hi, there"
print(c_s)              # the memory location has changed

In [ ]:
c_wchar_p(139966783348904)
>>> print(c_s.value)

In [ ]:
print(s)                # first object is unchanged


In [ ]:
p = create_string_buffer(3)            # create a 3 byte buffer, initialized to NUL bytes
print(sizeof(p), repr(p.raw))

In [ ]:
p = create_string_buffer(b"Hello")     # create a buffer containing a NUL terminated string
print(sizeof(p), repr(p.raw))

In [ ]:
print(repr(p.value))

In [ ]:
p = create_string_buffer(b"Hello", 10) # create a 10 byte buffer
print(sizeof(p), repr(p.raw))

In [ ]:
p.value = b"Hi"
print(sizeof(p), repr(p.raw))

In [ ]:
printf = libc.printf
printf(b"Hello, %s\n", b"World!")

In [ ]:
printf(b"Hello, %S\n", "World!")

In [ ]:
Hello, World!
14
>>> printf(b"Hello, %S\n", "World!")
Hello, World!
14
>>> printf(b"%d bottles of beer\n", 42)
42 bottles of beer
19
>>> printf(b"%f bottles of beer\n", 42.5)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ArgumentError: argument 2: exceptions.TypeError: Don't know how to convert parameter 2
>>>

In [ ]:
np.power(213/4, 0.5)

In [ ]:
0.17*12

In [ ]:
0.2*8

In [ ]:
5*39192/3

In [ ]:
712/5274

In [ ]:
51414/7703674

In [ ]:
5070/51414

In [ ]:
25*5/8

In [ ]:
4*6*9/3.62

In [ ]:
3*6*9/2.42

In [ ]:
2*7*9/4.84

In [ ]:
import datetime
import random
import matplotlib.pyplot as plt

# make up some data
x = [datetime.datetime.now() + datetime.timedelta(hours=i) for i in range(12)]
y = [i+random.gauss(0,1) for i,_ in enumerate(x)]

# plot
plt.bar(x,y)
# beautify the x-labels
plt.gcf().autofmt_xdate()

plt.show()

In [ ]:
df = pd.read_csv("/Users/mholman/Dropbox/misc/dates.csv")

In [ ]:
df2 = pd.read_csv("/Users/mholman/Dropbox/misc/funding.csv")

In [ ]:
df['d'] = df.date.apply(lambda x: pd.to_datetime(x))
df2['d'] = df2.date.apply(lambda x: pd.to_datetime(x))

In [ ]:
cumulative = np.cumsum(df.delta)

In [ ]:
labor=np.cumsum((cumulative+6)*(df.d-df.d.shift())/(365.25*np.timedelta64(1,'D')))

In [ ]:

import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(df.d, cumulative+6)

#ax2 = ax.twinx()

linestyle = (0, (1, 4))

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
#ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

panel = datetime.datetime(2015, 5, 5)
ax.axvline(panel, -10, 20, ls=linestyle, color='r')

prop = datetime.datetime(2016, 10, 28)
ax.axvline(prop, -10, 20, ls=linestyle, color='g')

funds = datetime.datetime(2017, 1, 27)
ax.axvline(funds, -10, 20, color='g')

ma = datetime.datetime(2017, 5, 29)
ax.axvline(ma, -10, 20, ls='--', color='black')

sk = datetime.datetime(2018, 8, 13)
ax.axvline(sk, -10, 20, ls='--', color='black')

mug1 = datetime.datetime(2017, 6, 22)
ax.axvline(mug1, -10, 20, ls='dotted', color='r')

mug2 = datetime.datetime(2017, 12, 18)
ax.axvline(mug2, -10, 20, ls='dotted', color='r')

mug3 = datetime.datetime(2018, 6, 28)
ax.axvline(mug3, -10, 20, ls='dotted', color='r')

neocp = datetime.datetime(2018, 11, 1)
ax.axvline(neocp, -10, 20, ls='solid', color='blue')

mug4 = datetime.datetime(2018, 12, 17)
ax.axvline(mug4, -10, 20, ls='dotted', color='r')

mug5 = datetime.datetime(2019, 7, 9)
ax.axvline(mug5, -10, 20, ls='dotted', color='r')

mug6 = datetime.datetime(2019, 12, 10)
ax.axvline(mug6, -10, 20, ls='dotted', color='r')

mug7 = datetime.datetime(2020, 6, 11)
ax.axvline(mug7, -10, 20, ls='dotted', color='r')

mug8 = datetime.datetime(2020, 12, 2)
ax.axvline(mug8, -10, 20, ls='dotted', color='r')

covid = datetime.datetime(2020, 3, 13)
ax.axvline(covid, -10, 20, ls='dashdot', color='purple')

vms = datetime.datetime(2019, 6, 6)
ax.axvline(vms, -10, 20, ls='dashdot', color='purple')

d_start = datetime.datetime(2018, 10, 1)
d_end = datetime.datetime(2019, 2, 10)
#ax.axhline(y=3, xmin=0.62, xmax=0.68, color='black')


dates = datetime.datetime(2016, 11, 1), datetime.datetime(2017, 11, 1), datetime.datetime(2018, 11, 1), datetime.datetime(2019, 11, 1), datetime.datetime(2020, 11, 1)
levels = 7, 7, 10, 11, 11
#ax.plot(dates, levels, color='gray')

ax.set_xlabel('Date')
ax.set_ylabel('MPC FTEs')


#ax2.set_ylabel('Total Funding ($M)')

funding = df2['total_funding']
d = df2['d']

linestyle = (0, (1, 4))

#ax2.xaxis_date() 
#ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
#ax2.plot(d, funding/1e6)

#ax2.plot(df.d, labor)

plt.savefig('/Users/mholman/Dropbox/misc/FTEs.png')

In [ ]:

import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(10, 6))
#ax.scatter(df.d, cumulative+6)

ax2 = ax.twinx()

linestyle = (0, (1, 4))

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

panel = datetime.datetime(2015, 5, 5)
ax.axvline(panel, -10, 20, ls=linestyle, color='r')

prop = datetime.datetime(2016, 10, 28)
ax.axvline(prop, -10, 20, ls=linestyle, color='g')

funds = datetime.datetime(2017, 1, 27)
ax.axvline(funds, -10, 20, color='g')

ma = datetime.datetime(2017, 5, 29)
#ax.axvline(ma, -10, 20, ls='--', color='black')

sk = datetime.datetime(2018, 8, 13)
#ax.axvline(sk, -10, 20, ls='--', color='black')

mug1 = datetime.datetime(2017, 6, 22)
ax.axvline(mug1, -10, 20, ls='dotted', color='r')

mug2 = datetime.datetime(2017, 12, 18)
ax.axvline(mug2, -10, 20, ls='dotted', color='r')

mug3 = datetime.datetime(2018, 6, 28)
ax.axvline(mug3, -10, 20, ls='dotted', color='r')

neocp = datetime.datetime(2018, 11, 1)
#ax.axvline(neocp, -10, 20, ls='solid', color='blue')

mug4 = datetime.datetime(2018, 12, 17)
ax.axvline(mug4, -10, 20, ls='dotted', color='r')

mug5 = datetime.datetime(2019, 7, 9)
ax.axvline(mug5, -10, 20, ls='dotted', color='r')

mug6 = datetime.datetime(2019, 12, 10)
ax.axvline(mug6, -10, 20, ls='dotted', color='r')

mug7 = datetime.datetime(2020, 6, 11)
ax.axvline(mug7, -10, 20, ls='dotted', color='r')

mug8 = datetime.datetime(2020, 12, 2)
ax.axvline(mug8, -10, 20, ls='dotted', color='r')

covid = datetime.datetime(2020, 3, 13)
#ax.axvline(covid, -10, 20, ls='dashdot', color='purple')

vms = datetime.datetime(2019, 6, 6)
#ax.axvline(vms, -10, 20, ls='dashdot', color='purple')

d_start = datetime.datetime(2018, 10, 1)
d_end = datetime.datetime(2019, 2, 10)
#ax.axhline(y=3, xmin=0.62, xmax=0.68, color='black')


dates = datetime.datetime(2016, 11, 1), datetime.datetime(2017, 11, 1), datetime.datetime(2018, 11, 1), datetime.datetime(2019, 11, 1), datetime.datetime(2020, 11, 1)
levels = 7, 7, 10, 11, 11
#ax.plot(dates, levels, color='gray')

ax.set_xlabel('Date')
ax.set_ylabel('MPC FTE-years')


ax2.set_ylabel('Funding ($M)')

funding = df2['total_funding']
coh_me = df2['coh_me']
expected = df2['expected']
d = df2['d']

linestyle = (0, (1, 4))

ax2.xaxis_date() 
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax2.plot(d, funding/1e6)
ax2.plot(d, coh_me/1e6, 'red')
ax2.plot(d, expected/1e6)

ax.plot(df.d, labor-9.7, 'black')
ax.set_ylim(-0.1, 40)
ax2.set_ylim(-0.1, 8)


plt.savefig('/Users/mholman/Dropbox/misc/funding.pdf')

In [ ]:
d_start

In [ ]:
plt.bar(df.d, cumulative+6)

In [ ]:
cumulative+6, df.d

In [ ]:
cumulative+6

In [ ]:
df.d

In [ ]:
df.diff

In [ ]:
import spicey as sp

In [ ]:
import wis

In [ ]:
(440e6-45.9e6)/(2*1.5e6*30)